In [1]:
import json

In [6]:
def load_pdf(hash_id):
    resources = json.load(open('./resources/res.json'))
    pdfname = list(filter(lambda x:x['hash'] == hash_id,resources))[0]['filename']
    return pdfname
load_pdf_name("857irwueifdkjwo9eruj2iroeakdfm")

'Monster-The-Investigative-Report.pdf'

In [8]:
from unstructured.partition.pdf import partition_pdf

ModuleNotFoundError: No module named 'pdf2image'